# **<a id="Content">HnM RecSys Notebook 9417</a>**

## **<a id="Content">Table of Contents</a>**
* [**<span>1. Imports</span>**](#Imports)  
* [**<span>2. Pre-Processing</span>**](#Pre-Processing)
* [**<span>3. Exploratory Data Analysis</span>**](#Exploratory-Data-Analysis)  
    * [**<span>3.1 Articles</span>**](#EDA::Articles)  
    * [**<span>3.2 Customers</span>**](#EDA::Customers)
    * [**<span>3.3 Transactions</span>**](#EDA::Transactions)
* [**<span>4. Helper FunctionsDecorators</span>**](#Helper-Functions)
* [**<span>5. Models</span>**](#Models) 
    * [**<span>5.1 Popularity</span>**](#Popularity-Model)   
    * [**<span>5.2 ALS</span>**](#Alternating-Least-Squares)  
    * [**<span>5.2 GBDT</span>**](#GBDT)  
    * [**<span>5.3 SGD/similar</span>**](#SGD)  
    * [**<span>5.4 NN</span>**](#NN)

## Imports

In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os
import re
import warnings
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
# import cudf # switch on P100 GPU for this to work in Kaggle
# import cupy as cp

# Importing data
articles = pd.read_csv('articles.csv')
print(articles.head())
print("--")
customers = pd.read_csv('customers.csv')
print(customers.head())
print("--")
transactions = pd.read_csv("transactions_train.csv")
print(transactions.head())
print("--")

   article_id  product_code          prod_name  product_type_no   
0   108775015        108775          Strap top              253  \
1   108775044        108775          Strap top              253   
2   108775051        108775      Strap top (1)              253   
3   110065001        110065  OP T-shirt (Idro)              306   
4   110065002        110065  OP T-shirt (Idro)              306   

  product_type_name  product_group_name  graphical_appearance_no   
0          Vest top  Garment Upper body                  1010016  \
1          Vest top  Garment Upper body                  1010016   
2          Vest top  Garment Upper body                  1010017   
3               Bra           Underwear                  1010016   
4               Bra           Underwear                  1010016   

  graphical_appearance_name  colour_group_code colour_group_name  ...   
0                     Solid                  9             Black  ...  \
1                     Solid               

## Pre-Processing

In [90]:
# ----- empty value stats -------------
print("Missing values: ")
print(customers.isnull().sum())
print("--\n")

print("FN Newsletter vals: ", customers['FN'].unique())
print("Active communication vals: ",customers['Active'].unique())
print("Club member status vals: ", customers['club_member_status'].unique())
print("Fashion News frequency vals: ", customers['fashion_news_frequency'].unique())
print("--\n")

# ---- data cleaning -------------

customers['FN'] = customers['FN'].fillna(0)
customers['Active'] = customers['Active'].fillna(0)

# replace club_member_status missing values with 'LEFT CLUB' --> no members with LEFT CLUB status in data
customers['club_member_status'] = customers['club_member_status'].fillna('LEFT CLUB')
customers['fashion_news_frequency'] = customers['fashion_news_frequency'].fillna('None')
customers['fashion_news_frequency'] = customers['fashion_news_frequency'].replace('NONE', 'None')
customers['age'] = customers['age'].fillna(customers['age'].mean())
customers['age'] = customers['age'].astype(int)
articles['detail_desc'] = articles['detail_desc'].fillna('None')


print("Customers' Missing values: ")
print(customers.isnull().sum())
print("--\n")

Missing values: 
customer_id                    0
FN                        895050
Active                    907576
club_member_status          6062
fashion_news_frequency     16011
age                        15861
postal_code                    0
dtype: int64
--

FN Newsletter vals:  [nan  1.]
Active communication vals:  [nan  1.]
Club member status vals:  ['ACTIVE' nan 'PRE-CREATE' 'LEFT CLUB']
Fashion News frequency vals:  ['NONE' 'Regularly' nan 'Monthly']
--

Customers' Missing values: 
customer_id               0
FN                        0
Active                    0
club_member_status        0
fashion_news_frequency    0
age                       0
postal_code               0
dtype: int64
--



In [91]:
# ---- memory optimizations -------------

# reference: https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65

# iterate through all the columns of a dataframe and reduce the int and float data types to the smallest possible size, ex. customer_id should not be reduced from int64 to a samller value as it would have collisions
import numpy as np
import pandas as pd

def reduce_mem_usage(df):
    """Iterate over all the columns of a DataFrame and modify the data type
    to reduce memory usage, handling ordered Categoricals"""
    
    # check the memory usage of the DataFrame
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type == 'category':
            if df[col].cat.ordered:
                # Convert ordered Categorical to an integer
                df[col] = df[col].cat.codes.astype('int16')
            else:
                # Convert unordered Categorical to a string
                df[col] = df[col].astype('str')
        
        elif col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min >= np.iinfo(np.int64).min and c_max <= np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    
    # check the memory usage after optimization
    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))

    # calculate the percentage of the memory usage reduction
    mem_reduction = 100 * (start_mem - end_mem) / start_mem
    print("Memory usage decreased by {:.1f}%".format(mem_reduction))
    
    return df

   

In [92]:
print("Articles Info: ")
print(articles.info())
print("Customer Info: ")
print(customers.info())
print("Transactions Info: ")
print(transactions.info())

Articles Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-n

In [93]:
# print unique values of customer columns
print("FN Newsletter vals: ", customers['FN'].unique())
print("Active communication vals: ",customers['Active'].unique())
print("Club member status vals: ", customers['club_member_status'].unique())
print("Fashion News frequency vals: ", customers['fashion_news_frequency'].unique())
print("--\n")

FN Newsletter vals:  [0. 1.]
Active communication vals:  [0. 1.]
Club member status vals:  ['ACTIVE' 'LEFT CLUB' 'PRE-CREATE']
Fashion News frequency vals:  ['None' 'Regularly' 'Monthly']
--



In [94]:
# explicitly convert club_member_status to ordinal values before mem optimization to avoid errors

customers['club_member_status'].replace({'LEFT CLUB': 0, 'PRE-CREATE': 1, 'ACTIVE': 2}, inplace=True)
customers['club_member_status'] = customers['club_member_status'].astype('int8')
print(customers['club_member_status'].unique())


[2 0 1]


In [95]:
# ---- memory optimizations -------------

# uses 8 bytes instead of given 64 byte string, reduces mem by 8x, 
# !!!! have to convert back before merging w/ sample_submissions.csv
# convert transactions['customer_id'] to 8 bytes int
# transactions['customer_id'] = transactions['customer_id'].astype('int64')
transactions['customer_id'] = transactions['customer_id'].apply(lambda x: int(x[-16:], 16)).astype('int64')
customers['customer_id'] = customers['customer_id'].apply(lambda x: int(x[-16:], 16)).astype('int64')

articles = reduce_mem_usage(articles)
customers = reduce_mem_usage(customers)
transactions = reduce_mem_usage(transactions)

# articles['article_id'] = articles['article_id'].astype('int32')
# transactions['article_id'] = transactions['article_id'].astype('int32') 
# # !!!! ADD LEADING ZERO BACK BEFORE SUBMISSION OF PREDICTIONS TO KAGGLE: 
# # Ex.: transactions['article_id'] = '0' + transactions.article_id.astype('str')

print("Articles Info: ")
print(articles.info())
print("Customer Info: ")
print(customers.info())
print("Transactions Info: ")
print(transactions.info())

Memory usage of dataframe is 20.13 MB
Memory usage after optimization is: 13.59 MB
Memory usage decreased by 32.5%
Memory usage of dataframe is 58.88 MB
Memory usage after optimization is: 39.25 MB
Memory usage decreased by 33.3%
Memory usage of dataframe is 1212.63 MB
Memory usage after optimization is: 697.26 MB
Memory usage decreased by 42.5%
Articles Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int32 
 1   product_code                  105542 non-null  int32 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int16 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  i

In [96]:
# print unique values of customer columns
print("FN Newsletter vals: ", customers['FN'].unique())
print("Active communication vals: ",customers['Active'].unique())
print("Club member status vals: ", customers['club_member_status'].unique())
print("Fashion News frequency vals: ", customers['fashion_news_frequency'].unique())
print("--\n")

FN Newsletter vals:  [0. 1.]
Active communication vals:  [0. 1.]
Club member status vals:  [2 0 1]
Fashion News frequency vals:  ['None' 'Regularly' 'Monthly']
--



In [97]:
# time-based splitting strategy

def split_train_val_data_and_drop_duplicates(transactions, days=7):
    """
    Splits the transaction training data into a training set and a validation set of 7 days to prevent data leakage.
    """
    
    transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])
    transactions = transactions.sort_values(by=['t_dat'])
    latest_transaction_date = transactions['t_dat'].max()
    
    training_set = transactions[transactions['t_dat'] < latest_transaction_date - pd.Timedelta(days=days)]
    validation_set = transactions[transactions['t_dat'] >= latest_transaction_date - pd.Timedelta(days=days)]
    
    print("Training set size:", len(training_set))
    print("Validation set size:", len(validation_set))
    print("Last date in training set:", training_set['t_dat'].max())
    print("Last date in validation set:", validation_set['t_dat'].max())

    # drop duplicate rows
    training_set = training_set.drop_duplicates().copy()
    validation_set = validation_set.drop_duplicates().copy()
    
    return training_set, validation_set

In [98]:
def preprocess_data(transactions_df, customers_df, articles_df, customers_col='customer_id', articles_col='article_id'):
    """
    Preprocesses customer and article IDs for use in a sparse matrix.
    
    Returns:
    - transactions_df: the input transaction DataFrame with two additional columns, 'user_index' and 'item_index',
                       that map customer and article IDs to their corresponding indices in a sparse matrix
    - customer_id_indices_map: a dictionary that maps customer IDs to their corresponding indices
    - article_id_indices_map: a dictionary that maps article IDs to their corresponding indices
    """
    # Create a list of unique customer IDs and product IDs
    all_customers = customers_df[customers_col].unique().tolist()
    all_articles = articles_df[articles_col].unique().tolist()

    # Create dicts mapping IDs to their corresponding indices
    customer_id_indices_map = {customer_id: i for i, customer_id in enumerate(all_customers)}
    article_id_indices_map = {article_id: i for i, article_id in enumerate(all_articles)}

    # Map customer and article IDs to their resp. indices in the transaction DataFrame
    transactions_df['user_index'] = transactions_df[customers_col].map(customer_id_indices_map)
    transactions_df['item_index'] = transactions_df[articles_col].map(article_id_indices_map)

    return transactions_df, all_customers, all_articles, customer_id_indices_map, article_id_indices_map

In [99]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,-6846340800584936,663713001,0.050842,2
1,2018-09-20,-6846340800584936,541518023,0.030487,2
2,2018-09-20,-8334631767138808638,505221004,0.015236,2
3,2018-09-20,-8334631767138808638,685687003,0.016937,2
4,2018-09-20,-8334631767138808638,685687004,0.016937,2


In [142]:
import numpy as np
from scipy.sparse import csr_matrix

# binary purchase interaction user-item matrix

def create_user_item_matrix(transactions):

    # Get unique user and item indices in asc. order
    user_indices = np.arange(transactions['user_index'].nunique())
    item_indices = np.arange(transactions['item_index'].nunique())

    # Create a dictionary mapping user and item indices to matrix indices
    user_index_dict = dict(zip(sorted(transactions['user_index'].unique()), user_indices))
    item_index_dict = dict(zip(sorted(transactions['item_index'].unique()), item_indices))

    # Create arrays of row indices, column indices, and data for the sparse matrix
    rows = []
    cols = []
    data = [] # purchased 1 or 0

    # Iterate over all possible combinations of user and item indices
    for user_index in user_indices:
        for item_index in item_indices:
            # Get the corresponding matrix indices for the user and item indices
            matrix_user_index = user_index_dict.get(user_index)
            matrix_item_index = item_index_dict.get(item_index)
            # Get the corresponding interaction value from the transactions dataframe
            interaction = transactions.loc[(transactions['user_index'] == user_index) & 
                                            (transactions['item_index'] == item_index), 'quantity'].values
            # Append the row index, column index, and interaction value to the corresponding arrays
            rows.append(matrix_user_index)
            cols.append(matrix_item_index)
            data.append(1 if len(interaction) > 0 else 0)

    # Create the sparse matrix using the row, column, and data arrays
    user_item_matrix = csr_matrix((data, (rows, cols)), shape=(len(user_indices), len(item_indices)))

    return user_item_matrix


In [101]:
from math import ceil

# calculate total number of transaction weeks in tranactions data
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])

# Compute the minimum and maximum date values
min_date = transactions['t_dat'].min()
max_date = transactions['t_dat'].max()

# Compute the number of weeks between the minimum and maximum date values
num_weeks = ceil((max_date - min_date).days / 7)

print(f"Total number of transaction weeks: {num_weeks}")


Total number of transaction weeks: 105


In [102]:
from datetime import datetime, timedelta

# only use last x weeks of transactions data since data is too large
def filter_transactions_last_x_weeks(transactions, x = 10):
    # Convert date strings to datetime objects
    transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])

    # Calculate the date x weeks ago from the latest transaction date
    latest_date = transactions['t_dat'].max()
    cutoff_date = latest_date - timedelta(weeks=x)

    # Filter transactions to only include those in the last x weeks
    filtered_transactions = transactions.loc[transactions['t_dat'] >= cutoff_date].copy()

    return filtered_transactions

In [103]:
def filter_customers_and_articles(customers, articles, filtered_transactions):
    # Get unique customer and article IDs from filtered transactions
    customer_ids = filtered_transactions['customer_id'].unique()
    article_ids = filtered_transactions['article_id'].unique()

    # Filter customers and articles to only include those in filtered transactions
    customers_filtered = customers.loc[customers['customer_id'].isin(customer_ids)].copy()
    articles_filtered = articles.loc[articles['article_id'].isin(article_ids)].copy()

    return customers_filtered, articles_filtered

## LightGBM

|Feature|LightGBM|XGBoost|CatBoost|
|:----|:----|:----|:----|
|Categoricals|Supports categorical features via one-hot encoding|Supports categorical features via one-hot encoding|Automatically handles categorical features using embeddings|
|Speed|Very fast training and prediction|Fast training and prediction|Slower than LightGBM and XGBoost|
|Handling Bias|Handles unbalanced classes via 'is_unbalance'|Handles unbalanced classes via 'scale_pos_weight'|Automatically handles unbalanced classes|
|Handling NaNs|Handles NaN values natively|Requires manual handling of NaNs|Automatically handles NaN values using special category|
|Custom Loss|Supports custom loss functions|Supports custom loss functions|Supports custom loss functions|


- Perform feature engineering using one-hot encoding or label encoding to encode the categorical features in the dataset.<br>
- Try different feature selection techniques, such as Recursive Feature Elimination (RFE) or SelectKBest, to select a smaller subset of features for the model.<br>
- Deal with class imbalance by adjusting the weights of the samples in the training set. Use the class_weights function from scikit-learn to calculate the weights based on the class distribution and pass them as - the weight parameter when creating the LightGBM datasets.<br>


- Use more advanced feature selection techniques such as feature importance analysis provided by LightGBM or PCA to reduce the dimensionality of the dataset and remove any multicollinearity. (??) <br> 
- Split the data into train and test sets using a time-based split based on the transaction date to avoid data leakage.<br>
  
- Train a LightGBM model on the training data using the selected features.<br>
  
- Experiment with different evaluation metrics to find the most appropriate one for your specific use case. For example, you could use the area under the ROC curve (AUC) or the F1-score if MAP does not perform well.<br>
- Use a time series cross-validation strategy to find the best hyperparameters for your model. This can be achieved using the TimeSeriesSplit function from scikit-learn instead of the default k-fold cross-validation.<br>
- Try different hyperparameters for the LightGBM model, such as the learning rate, number of estimators, max depth, etc., and use cross-validation to select the best combination of hyperparameters. (OR) <br> 
  - Bayesian optimization/Hyperopt to more efficiently search the hyperparameter space and find the optimal combination of hyperparameters.<br>
  
- Evaluate the model's performance on the val set using mean average precision (MAP) as the evaluation metric. <br>
- Once you have selected the best hyperparameters, train the final LightGBM model on the entire dataset using the selected features and hyperparameters.<br>
- Save the trained model for future use.<br>

To use LightGBM for a ranking problem, treat this as a binary classification problem where the target variable is whether an item is relevant or not to the user.

Then use LightGBM's ranking API, which is designed for ranking problems. Instead of optimizing for accuracy, the ranking API optimizes for ranking metric MAP. 

### Feature Engineering

In [104]:
# LightGBM imports

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import make_scorer
import lightgbm as lgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [105]:
# get top 200 customers by number of transactions
top_customers = transactions['customer_id'].value_counts().head(200).index.tolist()

# print num of transactions for the 200th customer
print(transactions['customer_id'].value_counts().sort_values(ascending=False).iloc[199])

# only get articles that were purchased by top 200 customers at least once in articles df
articles_top_200 = articles[articles['article_id'].isin(transactions[transactions['customer_id'].isin(top_customers)]['article_id'].unique())]

# only get 200 customers in customers df
customers_top_200 = customers[customers['customer_id'].isin(top_customers)]

articles = articles_top_200.copy()
customers = customers_top_200.copy()
transactions = transactions[transactions['customer_id'].isin(top_customers)].copy()
transactions = transactions.drop_duplicates().copy()

622


In [106]:
print(transactions.isnull().sum())
print(customers.isnull().sum())
print(articles.isnull().sum())

t_dat               0
customer_id         0
article_id          0
price               0
sales_channel_id    0
dtype: int64
customer_id               0
FN                        0
Active                    0
club_member_status        0
fashion_news_frequency    0
age                       0
postal_code               0
dtype: int64
article_id                      0
product_code                    0
prod_name                       0
product_type_no                 0
product_type_name               0
product_group_name              0
graphical_appearance_no         0
graphical_appearance_name       0
colour_group_code               0
colour_group_name               0
perceived_colour_value_id       0
perceived_colour_value_name     0
perceived_colour_master_id      0
perceived_colour_master_name    0
department_no                   0
department_name                 0
index_code                      0
index_name                      0
index_group_no                  0
index_group_name      

In [107]:
print(len(transactions))
print(len(customers))
print(len(articles))


126622
200
38919


In [108]:
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
5,110065011,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,12,Light Beige,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [109]:
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
38,1249760199313500820,1.0,1.0,2,Regularly,44,930b19ae7db8abb5a27f4da10217755a7305b4c452f5e0...
7066,3862718111684591643,0.0,0.0,2,None,30,7d81fea7e0b9c27deb7bd9c46304e24d176a6aa2ced422...
8859,-8098965676522405228,1.0,1.0,2,Regularly,23,84341fc1a4fff70f3effe1dc4de74167024562e00065b4...
9481,8346339317755757908,0.0,0.0,2,None,26,716a141c5fe7405d6ceab1ea281917fffca3c7a0e89314...
10095,-7779445982753353194,1.0,1.0,2,Regularly,26,d9953eb7a7a24998cec6deecc7ac24091ccc15e395ba12...


In [110]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
740,2018-09-20,1135991499650384534,668766002,0.042358,2
741,2018-09-20,1135991499650384534,652946001,0.050842,2
742,2018-09-20,1135991499650384534,691275008,0.067810,2
1260,2018-09-20,5085370976430926408,657476001,0.016937,2
1261,2018-09-20,5085370976430926408,685687003,0.016937,2


In [111]:
# Dropping columns with uninformative article data

articles = articles.drop(columns=['product_code', 'prod_name', 'product_type_name', 'product_group_name', 'graphical_appearance_name', 'department_name', 'index_name', 'index_group_name', 'section_name', 'garment_group_name', 'detail_desc'])
articles = articles.drop(columns=[col for col in articles.columns if 'colour_' in col or 'perceived_' in col])

In [112]:
articles.head()

,article_id,product_type_no,graphical_appearance_no,department_no,index_code,index_group_no,section_no,garment_group_no
0,108775015,253,1010016,1676,A,1,16,1002
1,108775044,253,1010016,1676,A,1,16,1002
3,110065001,306,1010016,1339,B,1,61,1017
4,110065002,306,1010016,1339,B,1,61,1017
5,110065011,306,1010016,1339,B,1,61,1017


These columns are left to capture any potential patterns in the other columns, such as how certain index codes or sections might be associated with higher or lower sales.

In [113]:
# Feature engineering
from sklearn.preprocessing import LabelEncoder

# Define mapping for fashion_news_frequency feature
fashion_news_freq_mapping = {'None': 0, 'Monthly': 1, 'Regularly': 2}

# label encode fashion_news_frequency feature
le = LabelEncoder()
customers['fashion_news_frequency'] = customers['fashion_news_frequency'].map(fashion_news_freq_mapping)
customers['fashion_news_frequency'] = le.fit_transform(customers['fashion_news_frequency'])

In [114]:
customers = customers.drop(['postal_code'], axis=1)
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age
38,1249760199313500820,1.0,1.0,2,1,44
7066,3862718111684591643,0.0,0.0,2,0,30
8859,-8098965676522405228,1.0,1.0,2,1,23
9481,8346339317755757908,0.0,0.0,2,0,26
10095,-7779445982753353194,1.0,1.0,2,1,26


In [115]:
# Feature engineering: encode nominal categorical features
ohe = OneHotEncoder()
# One-hot encode sales_channel_id feature
sales_channel_ohe = pd.get_dummies(transactions['sales_channel_id'], prefix='sales_channel')
transactions = pd.concat([transactions, sales_channel_ohe], axis=1)

# Drop the original sales_channel_id feature
transactions.drop('sales_channel_id', axis=1, inplace=True)

In [116]:
# boolean unique values of sales_channel_id after encoding
print(transactions['sales_channel_1'].unique())
print(transactions['sales_channel_2'].unique())

[False  True]
[ True False]


In [117]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_1,sales_channel_2
740,2018-09-20,1135991499650384534,668766002,0.042358,False,True
741,2018-09-20,1135991499650384534,652946001,0.050842,False,True
742,2018-09-20,1135991499650384534,691275008,0.067810,False,True
1260,2018-09-20,5085370976430926408,657476001,0.016937,False,True
1261,2018-09-20,5085370976430926408,685687003,0.016937,False,True


In [118]:
# Convert 't_dat' column to datetime format
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])

# Group by customer ID and find the first and last transaction dates
first_trans_dates = transactions.groupby('customer_id')['t_dat'].min().reset_index()
last_trans_dates = transactions.groupby('customer_id')['t_dat'].max().reset_index()

customer_purchase_engagement = pd.merge(first_trans_dates, last_trans_dates, on='customer_id', suffixes=('_first', '_last'))
# Create a new feature by calculating the time difference in days between first and last transactions
customer_purchase_engagement['time_diff_days'] = (customer_purchase_engagement['t_dat_last'] - customer_purchase_engagement['t_dat_first']).dt.days
# Drop the original first and last transaction date columns
customer_purchase_engagement.drop(['t_dat_first', 't_dat_last'], axis=1, inplace=True)
customer_purchase_engagement.head()

# Merge the customer_purchase_engagement dataframe with the customers dataframe
customers = pd.merge(customers, customer_purchase_engagement, on='customer_id', how='left')
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,time_diff_days
0,1249760199313500820,1.0,1.0,2,1,44,714
1,3862718111684591643,0.0,0.0,2,0,30,714
2,-8098965676522405228,1.0,1.0,2,1,23,727
3,8346339317755757908,0.0,0.0,2,0,26,724
4,-7779445982753353194,1.0,1.0,2,1,26,520


The above `time_diff_days` feature can potentially provide insights into a customer's engagement by looking at the gap in the number of days between the last purchase and the current date. <br> The assumption is, The larger the gap, the less engaged the customer is. 

In [119]:
# Join the transaction dataframe with the customers dataframe
merged = pd.merge(transactions, customers, on='customer_id', how='inner')

# Calculate the mean age for each article
item_mean_age = merged.groupby('article_id')['age'].mean()

# Calculate the difference between every user's age and the mean age of users who have purchased a particular item
merged['age_diff'] = merged['age'] - merged['article_id'].map(item_mean_age)

# Group by article and take the mean of age_diff
article_age_diff = merged.groupby('article_id')['age_diff'].mean()

# Append the age difference feature to the articles dataframe
articles['age_diff'] = articles['article_id'].map(article_age_diff)

articles.head()

,article_id,product_type_no,graphical_appearance_no,department_no,index_code,index_group_no,section_no,garment_group_no,age_diff
0,108775015,253,1010016,1676,A,1,16,1002,2.583792e-15
1,108775044,253,1010016,1676,A,1,16,1002,0.000000e+00
3,110065001,306,1010016,1339,B,1,61,1017,-1.421085e-15
4,110065002,306,1010016,1339,B,1,61,1017,0.000000e+00
5,110065011,306,1010016,1339,B,1,61,1017,1.937844e-15


Mean age_diff for every article. It can be useful for predicting whether a user will buy an item based on their age and the age of other users who have already bought the same item. 

Intuituion behind the `age_diff` feature:

Let's say we have a dataset of customers who made transactions for a particular item with article_id = 123. Here is an example of how we can calculate the age_diff feature: <br>
Assume that the mean age of all customers who bought the item with article_id = 123 is 40 years old <br>
Customer A made a transaction for item with article_id = 123 and their age is 35. The age_diff feature for this transaction would be -5. (35 - 40). <br>
Customer B made a transaction for item with article_id = 123 and their age is 50. The age_diff feature for this transaction would be 10. (50 - 40). <br>
Customer C made a transaction for item with article_id = 123 and their age is 40. The age_diff feature for this transaction would be 0. (40 - 40). <br>
So, the age_diff feature measures the difference between the age of each customer who bought a specific item and the average age of all customers who bought that item. <br>

Therefore, the age_diff is the mean of all these individual age_diff values for each customer who bought the item with article_id = 123. age_diff = -1.66 for this example<br>


In [120]:
# Calculate mean, max, and min age for each item
item_mean_age = merged.groupby('article_id')['age'].mean()
item_max_age = merged.groupby('article_id')['age'].max()
item_min_age = merged.groupby('article_id')['age'].min()

# Merge the features back into the articles dataframe
articles = articles.merge(item_mean_age, on='article_id', how='left')
articles = articles.merge(item_max_age, on='article_id', how='left')
articles = articles.merge(item_min_age, on='article_id', how='left')

# Rename the columns to make them more descriptive
articles = articles.rename(columns={'age_x': 'mean_purchase_age', 'age_y': 'max_purchase_age', 'age': 'min_purchase_age'})

articles.head()

,article_id,product_type_no,graphical_appearance_no,department_no,index_code,index_group_no,section_no,garment_group_no,age_diff,mean_purchase_age,max_purchase_age,min_purchase_age
0,108775015,253,1010016,1676,A,1,16,1002,2.583792e-15,37.409091,61,24
1,108775044,253,1010016,1676,A,1,16,1002,0.000000e+00,37.000000,51,22
2,110065001,306,1010016,1339,B,1,61,1017,-1.421085e-15,43.600000,55,23
3,110065002,306,1010016,1339,B,1,61,1017,0.000000e+00,50.000000,50,50
4,110065011,306,1010016,1339,B,1,61,1017,1.937844e-15,46.181818,59,29


Intuituion behind the `*_purchase_age` feature:

Additional age features to capture more information about the age of the customers who bought the respective articles. The gbdt might be able to learn more complex patterns from these features. <br>

In [121]:
# Calculate purchased item count for each user
transactions['quantity'] = 1
user_item_count = transactions.groupby(['customer_id', 'article_id'])['quantity'].sum().reset_index()

# Calculate total item count for each article
total_item_count = transactions.groupby('article_id')['quantity'].sum().reset_index()
total_item_count.columns = ['article_id', 'total_items']

user_item_count = pd.merge(user_item_count, total_item_count, on='article_id', how='left')

# Calculate ratio of purchased item count and total item count
user_item_count['article_engagement_ratio'] = user_item_count['quantity'] / user_item_count['total_items']


transactions = pd.merge(transactions, user_item_count[['customer_id', 'article_id', 'article_engagement_ratio']], on=['customer_id', 'article_id'], how='left')
transactions['quantity'] = user_item_count['quantity']

# fill missing values with 0
transactions['quantity'] = transactions['quantity'].fillna(0)
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_1,sales_channel_2,quantity,article_engagement_ratio
0,2018-09-20,1135991499650384534,668766002,0.042358,False,True,1.0,1.000000
1,2018-09-20,1135991499650384534,652946001,0.050842,False,True,1.0,1.000000
2,2018-09-20,1135991499650384534,691275008,0.067810,False,True,1.0,1.000000
3,2018-09-20,5085370976430926408,657476001,0.016937,False,True,1.0,0.500000
4,2018-09-20,5085370976430926408,685687003,0.016937,False,True,1.0,0.166667


Intuition behind feature: <br>

`article_engagement_ratio`: The feature is ratio of one user's purchased item count and the item's total purchase count. This serves to measure how engaged a user is with a particular item, which can be useful for predicting whether a user will buy similar items. <br>
Can also be used to measure how popular an item is, and can be used to potentially diversify recommendations.

In [122]:
transactions, all_customers, all_articles, customer_id_indices_map, article_id_indices_map = preprocess_data(transactions, customers, articles)

print("Total num of customers: ", len(all_customers))
print("Total num of articles: ", len(all_articles))
print("Customer ID mapping: ", list(customer_id_indices_map.items())[:5])
print("Article ID mapping: ", list(article_id_indices_map.items())[:5])
transactions.head()

Total num of customers:  200
Total num of articles:  38919
Customer ID mapping:  [(1249760199313500820, 0), (3862718111684591643, 1), (-8098965676522405228, 2), (8346339317755757908, 3), (-7779445982753353194, 4)]
Article ID mapping:  [(108775015, 0), (108775044, 1), (110065001, 2), (110065002, 3), (110065011, 4)]


,t_dat,customer_id,article_id,price,sales_channel_1,sales_channel_2,quantity,article_engagement_ratio,user_index,item_index
0,2018-09-20,1135991499650384534,668766002,0.042358,False,True,1.0,1.000000,5,11563
1,2018-09-20,1135991499650384534,652946001,0.050842,False,True,1.0,1.000000,5,9899
2,2018-09-20,1135991499650384534,691275008,0.067810,False,True,1.0,1.000000,5,14438
3,2018-09-20,5085370976430926408,657476001,0.016937,False,True,1.0,0.500000,10,10307
4,2018-09-20,5085370976430926408,685687003,0.016937,False,True,1.0,0.166667,10,13608


In [137]:
# user item matrix -- rows are users, columns are items, doesnt need article and customer data

user_item_matrix = create_user_item_matrix(transactions)
user_item_matrix 

<200x38919 sparse matrix of type '<class 'numpy.intc'>'
	with 7783800 stored elements in Compressed Sparse Row format>

In [149]:
print(user_item_matrix[:10, :10].toarray())

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [138]:
from implicit.als import AlternatingLeastSquares
from sklearn.metrics.pairwise import cosine_similarity

# from the als_strat1_hyperparam_log 
# Create ALS model with default parameters
alpha = 25
als_model = AlternatingLeastSquares(factors=55, iterations=20, regularization=0.18)

# Fit model to user-item matrix
als_model.fit(user_item_matrix*alpha)

# Latent factors matrices
item_factors = als_model.item_factors
user_factors = als_model.user_factors

# item-item cosine similarity 
item_similarities = cosine_similarity(item_factors, dense_output=False)
# user-user cosine similarity
user_similarities = cosine_similarity(user_factors, dense_output=False)

k = 5
# Get top-k most similar items for each item
top_k_similar_items = item_similarities.argsort()[:, -k-1:-1]
# Get top-k most similar user for each user
top_k_similar_users = user_similarities.argsort()[:, -k-1:-1]

100%|██████████| 20/20 [00:24<00:00,  1.21s/it]


In [139]:
print(item_similarities.shape)
print(user_similarities.shape)

(38919, 38919)
(200, 200)


In [140]:
# important: add user_index and item_index to customers and articles respectively
customers['user_index'] = customers['customer_id'].map(customer_id_indices_map)
articles['item_index'] = articles['article_id'].map(article_id_indices_map)


In [146]:
user_indices = np.arange(transactions['user_index'].nunique())
item_indices = np.arange(transactions['item_index'].nunique())
user_index_dict = dict(zip(sorted(transactions['user_index'].unique()), user_indices))
item_index_dict = dict(zip(sorted(transactions['item_index'].unique()), item_indices))

# print first 5 key and values in the dictionary
print(list(user_index_dict.items())[:5])
print(list(item_index_dict.items())[:5])


[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]


In [160]:
# Create a mapping from user IDs to matrix indices
user_indices = np.arange(transactions['user_index'].nunique())
item_indices = np.arange(transactions['item_index'].nunique())
user_index_dict = dict(zip(sorted(transactions['user_index'].unique()), user_indices))
item_index_dict = dict(zip(sorted(transactions['item_index'].unique()), item_indices))


# Create features for each user based on the average quantity purchased by similar customers
for i in range(len(customers)):
    customer_id = customers.loc[i, 'user_index']
    # Get the matrix index for the current customer
    customer_idx = user_index_dict.get(customer_id, -1)
    
    if customer_idx != -1:
        similar_user_idxs = top_k_similar_users[customer_idx]
        # Compute the mean of the user-item matrix for the similar users
        user_feature = user_item_matrix[similar_user_idxs, :].mean(axis=0).A1
        
        # Store the mean in the customers DataFrame
        customers.loc[i, 'user_purchase_quant'] = user_feature.mean()
    else:
        # If the current customer is not in the user-item matrix, set the feature to NaN
        customers.loc[i, 'user_purchase_quant'] = np.nan

# Print the head of the customers DataFrame
customers.head()


,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,time_diff_days,user_index,user_purchase_quant
0,1249760199313500820,1.0,1.0,2,1,44,714,0,0.020931
1,3862718111684591643,0.0,0.0,2,0,30,714,1,0.017112
2,-8098965676522405228,1.0,1.0,2,1,23,727,2,0.014913
3,8346339317755757908,0.0,0.0,2,0,26,724,3,0.017267
4,-7779445982753353194,1.0,1.0,2,1,26,520,4,0.017842


Intuition behind feature: <br>

`user_purchase_quant`: Gets the average quantity of items purchased by the k most similar customers to that customer. It looks at what other customers who are similar to this customer have bought and calculates the average amount of each item they bought. This feature can be used to predict what items a customer is likely to buy in the future based on what similar customers have bought in the past. This feature aims to capture purchase behaviours of a customer.<br>

For example, if a customer typically buys a lot of bomber jackets, and the top k most similar customers to that customer also tend to buy a lot of bomber jackets, then the average quantity of bomber jackets purchased by those similar customers could be a good predictor of how much the original customer is likely to purchase in the future. This however assumes that the k most similar customers have similar purchase behaviours to the customers in question, and on its own is not a strong feature.<br>


In [175]:
# Create a binary feature for each item that indicates whether or not a customer has bought that item,
# based on whether other customers who bought similar items also tended to buy that item.
for i in range(len(articles)):
    item_id = articles.loc[i, 'item_index']
    # Get the matrix index for the current item
    item_idx = item_index_dict.get(item_id, -1)
    
    if item_idx != -1:
        # List of (column) indices of similar items
        similar_items = top_k_similar_items[item_idx]
        
        # Find the customers who have purchased the current item
        customer_indices = np.where(user_item_matrix[:, item_idx].toarray()[:, 0] == 1)[0]
        
        # Binary vector representing the customer's purchases for the similar items
        customer_purchases = user_item_matrix[customer_indices, :][:, similar_items].toarray()
        article_preference = np.any(customer_purchases, axis=0)
        
        # Set article_preference to 1 if any customer has purchased the item, 0 otherwise
        articles.loc[i, 'article_preference'] = int(np.any(article_preference))
    else:
        articles.loc[i, 'article_preference'] = np.nan

In [178]:
articles[articles['article_preference'] == 0].head()

,article_id,product_type_no,graphical_appearance_no,department_no,index_code,index_group_no,section_no,garment_group_no,age_diff,mean_purchase_age,max_purchase_age,min_purchase_age,item_index,article_preference
6,111586001,273,1010016,3608,B,1,62,1021,0.0,50.0,51,49,6,0.0
28,145872053,252,1010010,8559,S,26,22,1005,0.0,43.0,43,43,28,0.0
57,181448103,302,1010017,3937,D,2,51,1017,0.0,28.0,28,28,57,0.0
89,189654045,275,1010010,1643,D,2,51,1002,0.0,30.0,32,28,89,0.0
103,201219003,304,1010010,3608,B,1,62,1021,0.0,54.0,54,54,103,0.0


Intuition behind feature: <br>

`article_preference`: Binary feature for each item that indicates whether or not a customer has bought that item, based on whether other customers who bought similar items also tended to buy that item. <br>

This feature can be useful for a fashion-based recommender system because it captures the idea that customers who have similar tastes or preferences tend to buy similar items. For example, if a customer has a history of buying shirts and other customers who bought similar shorts also tended to buy a specific pair of jeans, then the binary feature for those jeans would be set to 1 for that customer, indicating that they are likely to be interested in those shoes. The binary feature can then be used as a predictor for which items to recommend to the customer.

In [181]:
# Create feature for each item based on the total number of times it was purchased by similar customers
for i in range(len(articles)):
    item_id = articles.loc[i, 'item_index']
    item_idx = item_index_dict.get(item_id, -1)
    
    if item_idx != -1:
    
        similar_items = top_k_similar_items[item_idx]
        
        # Find the customers who have purchased the current item
        customer_indices = np.where(user_item_matrix[:, item_idx].toarray()[:, 0] == 1)[0]
        
        # Compute the mean of the user-item matrix for the similar items
        item_feature = user_item_matrix[customer_indices, :][:, similar_items].sum() / len(customer_indices)
        articles.loc[i, 'item_purchase_frequency'] = item_feature.mean()
        
    else:
        articles.loc[i, 'item_purchase_frequency'] = np.nan

In [182]:
articles.head()

,article_id,product_type_no,graphical_appearance_no,department_no,index_code,index_group_no,section_no,garment_group_no,age_diff,mean_purchase_age,max_purchase_age,min_purchase_age,item_index,article_preference,item_purchase_frequency
0,108775015,253,1010016,1676,A,1,16,1002,2.583792e-15,37.409091,61,24,0,1.0,0.357143
1,108775044,253,1010016,1676,A,1,16,1002,0.000000e+00,37.000000,51,22,1,1.0,0.615385
2,110065001,306,1010016,1339,B,1,61,1017,-1.421085e-15,43.600000,55,23,2,1.0,1.250000
3,110065002,306,1010016,1339,B,1,61,1017,0.000000e+00,50.000000,50,50,3,1.0,5.000000
4,110065011,306,1010016,1339,B,1,61,1017,1.937844e-15,46.181818,59,29,4,1.0,0.666667


Intuition behind feature: <br>

`item_purchase_frequency`: It gives an estimate of how frequently an item is being bought by customers who have similar purchase histories. This feature is useful because it can provide insights into purchasing patterns and identify popular items that are often bought together. <br> It can potentially help identify popular items among customer groups, and the lightgbm model can potentially use this feature.

In [194]:
merged_df = pd.merge(transactions, articles[['item_index']], on='item_index', how='left')

merged_df.head()

,t_dat,customer_id,article_id,price,sales_channel_1,sales_channel_2,quantity,article_engagement_ratio,user_index,item_index
0,2018-09-20,1135991499650384534,668766002,0.042358,False,True,1.0,1.000000,5,11563
1,2018-09-20,1135991499650384534,652946001,0.050842,False,True,1.0,1.000000,5,9899
2,2018-09-20,1135991499650384534,691275008,0.067810,False,True,1.0,1.000000,5,14438
3,2018-09-20,5085370976430926408,657476001,0.016937,False,True,1.0,0.500000,10,10307
4,2018-09-20,5085370976430926408,685687003,0.016937,False,True,1.0,0.166667,10,13608


In [196]:
# Merge transactions with articles on item_index to access the price of each article
merged_df = pd.merge(transactions, articles[['item_index']], on='item_index', how='left')

# Compute the average price levels of all articles purchased by similar customers who have purchased this particular article in the past
for i in range(len(articles)):
    item_id = articles.loc[i, 'item_index']
    item_idx = item_index_dict.get(item_id, -1)
    
    if item_idx != -1:
        similar_items = top_k_similar_items[item_idx]
        
        # Find the customers who have purchased the current item
        customer_indices = np.where(user_item_matrix[:, item_idx].toarray()[:, 0] == 1)[0]
        
        # Compute the average price levels of all articles purchased by similar customers who have purchased this particular article in the past
        item_feature = merged_df.loc[(merged_df['item_index'] == item_id) & (merged_df['user_index'].isin(customer_indices)), 'price'].mean()
        articles.loc[i, 'item_avg_price_level'] = item_feature
    else:
        articles.loc[i, 'item_avg_price_level'] = np.nan

Intuition behind feature: <br>

`item_avg_price_level`: Calculates the average price levels of all articles purchased by similar customers who have purchased this particular article in the past.  <br> It provides information on the typical price level of articles that are purchased together with a given article, as indicated by the purchasing patterns of similar customers. 

For example, if customers who frequently purchase articles A also tend to purchase higher-priced article, then the "item_avg_price_level" feature for article A would be relatively high. 

In [197]:
articles.head()

,article_id,product_type_no,graphical_appearance_no,department_no,index_code,index_group_no,section_no,garment_group_no,age_diff,mean_purchase_age,max_purchase_age,min_purchase_age,item_index,article_preference,item_purchase_frequency,item_avg_price_level
0,108775015,253,1010016,1676,A,1,16,1002,2.583792e-15,37.409091,61,24,0,1.0,0.357143,0.007713
1,108775044,253,1010016,1676,A,1,16,1002,0.000000e+00,37.000000,51,22,1,1.0,0.615385,0.007809
2,110065001,306,1010016,1339,B,1,61,1017,-1.421085e-15,43.600000,55,23,2,1.0,1.250000,0.025421
3,110065002,306,1010016,1339,B,1,61,1017,0.000000e+00,50.000000,50,50,3,1.0,5.000000,0.024124
4,110065011,306,1010016,1339,B,1,61,1017,1.937844e-15,46.181818,59,29,4,1.0,0.666667,0.016663


In [199]:
# # print num of unique articles['graphical_appearance_no']
print(articles['graphical_appearance_no'].nunique())
print(articles['product_type_no'].nunique())
print(articles['department_no'].nunique())
print(articles['index_code'].nunique())
print(articles['index_group_no'].nunique()) 
print(articles['section_no'].nunique())
print(articles['garment_group_no'].nunique())

30
114
288
10
5
57
21


In [208]:
# drop customer_id and article_id 
articles_final_df = articles.drop(['article_id'], axis=1, inplace=True).copy()
customers_final_df = customers.drop(['customer_id'], axis=1, inplace=True).copy()
transactions_final_df = transactions.drop(['article_id', 'customer_id'], axis=1, inplace=True).copy()

# Merge transactions with customers
df = pd.merge(transactions_final_df, customers_final_df, on='user_index', how='left')

# Merge resulting dataframe with articles
df = pd.merge(df, articles_final_df, on='item_index', how='left')


KeyError: "['article_id'] not found in axis"

In [210]:
df.head()

,t_dat,customer_id_x,article_id_x,price,sales_channel_1,sales_channel_2,quantity,article_engagement_ratio,user_index,item_index,...,index_group_no,section_no,garment_group_no,age_diff,mean_purchase_age,max_purchase_age,min_purchase_age,article_preference,item_purchase_frequency,item_avg_price_level
0,2018-09-20,1135991499650384534,668766002,0.042358,False,True,1.0,1.000000,5,11563,...,1,6,1010,0.0,51.0,51,51,0.0,0.000000,0.042358
1,2018-09-20,1135991499650384534,652946001,0.050842,False,True,1.0,1.000000,5,9899,...,1,57,1016,0.0,51.0,51,51,0.0,0.000000,0.050842
2,2018-09-20,1135991499650384534,691275008,0.067810,False,True,1.0,1.000000,5,14438,...,1,18,1010,0.0,51.0,51,51,0.0,0.000000,0.067810
3,2018-09-20,5085370976430926408,657476001,0.016937,False,True,1.0,0.500000,10,10307,...,2,53,1005,0.0,58.0,67,49,1.0,3.000000,0.016937
4,2018-09-20,5085370976430926408,685687003,0.016937,False,True,1.0,0.166667,10,13608,...,1,15,1023,0.0,42.0,67,29,1.0,0.833333,0.019760


### Model Training

In [ ]:
# Prepare the data
transactions['purchased'] = 1
customers_articles = pd.merge(customers, articles, on=None, how='outer')

# left join with transaction data to get purchase history
df = pd.merge(customers_articles, transactions, on=['customer_id', 'article_id'], how='left')
df['purchased'] = df['quantity'].fillna(0)

# quantity not really needed anymore
df = df.drop(['quantity'], axis=1)

# fill NaN values with 0 for other columns -- Lightgbm will handle this though
df = df.fillna(0)

df.head()

In [ ]:
# Split the data into training and testing sets
training_set, validation_set = split_train_val_data_by_time_and_drop_duplicates(transactions)

X_train = training_set.drop(['purchased', 'article_id', 'customer_id'], axis=1)
y_train = training_set['purchased']

X_test = validation_set.drop(['purchased', 'article_id', 'customer_id'], axis=1)
y_test = validation_set['purchased']

In [ ]:
training_set.head()

In [ ]:
validation_set.head()

In [ ]:
user_item_training_matrix = create_user_item_matrix(training_set)
user_item_validation_matrix = create_user_item_matrix(validation_set)

user_item_training_matrix 

In [ ]:
# check for duplicates
print(training_set.duplicated().sum())
print(validation_set.duplicated().sum())

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import average_precision_score

X_train = training_set.drop(['purchased', 'article_id', 'customer_id'], axis=1)
y_train = training_set['purchased']

X_test = validation_set.drop(['purchased', 'article_id', 'customer_id'], axis=1)
y_test = validation_set['purchased']

# Train a LightGBM model
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

clf = lgb.LGBMClassifier(**params)
grid_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'num_leaves': [15, 31, 63],
    'min_child_samples': [20, 30, 40]
}

grid_search = GridSearchCV(clf, grid_params, cv=5, scoring='average_precision', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Evaluate the model
y_pred = grid_search.predict_proba(X_test)[:, 1]
map_score = average_precision_score(y_test, y_pred, pos_label=1, average='weighted')

# Generate recommendations for a given customer
def get_top_articles(customer_id):
    articles['prob'] = grid_search.predict_proba(X)[:, 1]
    recommendations = articles.sort_values('prob', ascending=False).head(12)
    return recommendations

# Print the MAP@12 score and some sample recommendations for a customer
print('MAP@12 score:', map_score)
print('Sample recommendations for customer 123:', get_top_articles(123))


# todo


- create a purchased variable for customer x article (1 if purchased, 0 if not purchased)
- Merge datasets on user_index and article_index and drop customer_id and article_id (also check for duplicates)
- Cosine similarity features calc for Lightgbm
- Lightgbm training with training with MAP as eval metric, grid search for hyperparams (ref. kaggle for starting params) (in built train test split? or by dates?)
- Lightgbm recommendation example
  

- Baseline model evalutaion for top 200 (same train test split as Lightgbm)

- Redo ALS for top 200 (same train test split as Lightgbm)
- comparisonm of ALS and Lightgbm and baseline model